In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

import tensorflow as tf
from tensorflow import keras
import keras
from tensorflow.keras.models import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.layers import *
from tensorflow.keras.losses import MeanSquaredError

import numpy as np
import pandas as pd
import cvxpy as cp
from tqdm import tqdm

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
cwd = os.getcwd()

def make_dir(path):
    if os.path.exists(path) is False:
        os.makedirs(path)

def evaluate_prediction(predictions, actual, model_name):
    errors = predictions - actual
    mse = np.square(errors).mean()
    rmse = np.sqrt(mse)
    mae = np.abs(errors).mean()
    print('MAE: {:.2f}'.format(mae))
    print('RMSE: {:.2f}'.format(rmse))
    print('')
    print('')
    return mae, rmse

In [3]:
sl = pd.read_csv('dataset/SCED (5m).csv')
sl['sced_time_stamp_local'] = pd.to_datetime(sl['sced_time_stamp_local'])
sl.set_index('sced_time_stamp_local', inplace=True)
sl = sl.resample('h').mean()
date_range = pd.date_range(start=sl.index.min(), end=sl.index.max(), freq='h')
sl = sl[~sl.index.duplicated(keep='first')]
sl = sl.reindex(date_range, fill_value=np.nan)
sl.interpolate(method='time', inplace=True)


In [4]:
dap = pd.read_csv('dataset/DAP (1hr).csv')
dap['timestamp'] = pd.to_datetime(dap['timestamp'])
dap.set_index('timestamp', inplace=True)
date_range = pd.date_range(start=dap.index.min(), end='2024-01-01 23:55:00', freq='h')
dap = dap[~dap.index.duplicated(keep='first')]
dap = dap.reindex(date_range, fill_value=np.nan)
dap.interpolate(method='time', inplace=True)

/var/folders/kv/ldp2ch2s1md4v31v75w2dt2c0000gn/T/ipykernel_48456/1171035437.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dap['timestamp'] = pd.to_datetime(dap['timestamp'])


In [5]:
forecasted_tlf = pd.read_csv('dataset/FORECASTED_TLF (15m).csv')
forecasted_tlf['datetime'] = pd.to_datetime(forecasted_tlf['datetime'])
forecasted_tlf.set_index('datetime', inplace=True)
date_range = pd.date_range(start=forecasted_tlf.index.min(), end='2024-01-01 23:55:00', freq='h')
forecasted_tlf = forecasted_tlf[~forecasted_tlf.index.duplicated(keep='first')]
forecasted_tlf = forecasted_tlf.reindex(date_range, fill_value=np.nan)
forecasted_tlf.interpolate(method='time', inplace=True)


In [6]:
#prepare the data for training and testing
df = pd.concat([dap, sl, forecasted_tlf], axis=1)
df.columns = ['DAP', 'SCED', 'F_TLF']
df.dropna(inplace=True)

df

,DAP,SCED,F_TLF
2019-01-02 00:00:00,23.9250,26.216109,0.016499
2019-01-02 01:00:00,23.3140,25.671136,0.016441
2019-01-02 02:00:00,23.3475,24.970952,0.016411
2019-01-02 03:00:00,23.0595,24.787712,0.016441
2019-01-02 04:00:00,25.2672,24.887786,0.016534
...,...,...,...
2023-12-31 19:00:00,19.3715,14.422107,0.021356
2023-12-31 20:00:00,16.7739,12.597410,0.021367
2023-12-31 21:00:00,15.0035,10.170172,0.021383
2023-12-31 22:00:00,15.7699,9.872549,0.021408


In [7]:

## log transform
log_data = df.copy(deep=True)
log_data.loc[:,"SCED"] = np.log(df.loc[:,"SCED"] + 1 - min(df.loc[:,"SCED"]))
log_data.loc[:,"DAP"] = np.log(df.loc[:,"DAP"] + 1 - min(df.loc[:,"DAP"]))
log_data.loc[:,"F_TLF"] = np.log(df.loc[:,"F_TLF"] + 1 - min(df.loc[:,"F_TLF"]))

#Shift for prediction
df['DAP'] = df['DAP'].shift(-24)

# 3 years training, 1 year validation, 1 year testing
x_train_df_reg = log_data.loc[:'2021-12-31 23:55:00'].iloc[:,:]
x_val_df_reg = log_data.loc['2022-01-01 00:00:00':'2022-12-31 23:55:00'].iloc[:,:]
x_test_df_reg = log_data.loc['2023-01-01 00:00:00':].iloc[:,:]

# Shift back for target variable
df['DAP'] = df['DAP'].shift(24)

#Change target variable to DAP
y_train_df_reg = log_data.loc[:'2021-12-31 23:55:00'].iloc[:, :1]
y_val_df_reg = log_data.loc['2022-01-01 00:00:00':'2022-12-31 23:55:00'].iloc[:, :1]
y_test_df_reg = log_data.loc['2023-01-01 00:00:00':].iloc[:, :1]

x_train_df_reg.reset_index(drop=True, inplace=True)
x_val_df_reg.reset_index(drop=True, inplace=True)
x_test_df_reg.reset_index(drop=True, inplace=True)

# Standardization
x_mean_reg, x_std_reg = x_train_df_reg.mean(), x_train_df_reg.std()
y_mean_reg, y_std_reg = y_train_df_reg.mean(), y_train_df_reg.std()

x_std_reg = x_std_reg +0.00001

x_train_reg = (x_train_df_reg - x_mean_reg)/x_std_reg
x_val_reg = (x_val_df_reg - x_mean_reg)/x_std_reg
x_test_reg = (x_test_df_reg - x_mean_reg)/x_std_reg

y_train_reg = (y_train_df_reg - y_mean_reg)/y_std_reg
y_val_reg = (y_val_df_reg - y_mean_reg)/y_std_reg
y_test_reg = (y_test_df_reg - y_mean_reg)/y_std_reg

# Shift the data for the lags
n_steps_in = 24
n_steps_out = 24

x_train_lstm = np.array([x_train_reg[i:i+n_steps_in] for i in range(0, x_train_reg.shape[0]-n_steps_in-n_steps_out+1)])
y_train_lstm = np.array([y_train_reg[i+n_steps_in:i+n_steps_in+n_steps_out] for i in range(0, y_train_reg.shape[0]-n_steps_in-n_steps_out+1)])

x_val_lstm = np.array([x_val_reg[i:i+n_steps_in] for i in range(0, x_val_reg.shape[0]-n_steps_in-n_steps_out+1)])
y_val_lstm = np.array([y_val_reg[i+n_steps_in:i+n_steps_in+n_steps_out] for i in range(0, y_val_reg.shape[0]-n_steps_in-n_steps_out+1)])

x_test_lstm = np.array([x_test_reg[i:i+n_steps_in] for i in range(0, x_test_reg.shape[0]-n_steps_in-n_steps_out+1)])
y_test_lstm = np.array([y_test_reg[i+n_steps_in:i+n_steps_in+n_steps_out] for i in range(0, y_test_reg.shape[0]-n_steps_in-n_steps_out+1)])

print(x_train_lstm.shape,y_train_lstm.shape,x_val_lstm.shape,y_val_lstm.shape, x_test_lstm.shape,y_test_lstm.shape)

(26233, 24, 3) (26233, 24, 1) (8713, 24, 3) (8713, 24, 1) (8713, 24, 3) (8713, 24, 1)


In [8]:
# set hyperparameters
n_neurons  = 64  # number of neurons in the Dense layer
activation     = 'relu' # activation function
learning_rate  = 0.0005
minibatch_size = 64
num_epochs     = 50

# MLP model
lstm_model = Sequential()
lstm_model.add(LSTM(n_neurons,input_shape=(x_train_lstm.shape[1],x_train_lstm.shape[2]),
               return_sequences=True,activation=activation))
lstm_model.add(LSTM(n_neurons,return_sequences=False,
               activation=activation))
lstm_model.add(Dense(n_neurons,activation=activation))
lstm_model.add(Dense(y_train_lstm.shape[-2],activation='linear')) 

lstm_model.compile(loss='mse',optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate))

lstm_model.summary()

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

history = lstm_model.fit(x_train_lstm, y_train_lstm, 
                        validation_data = (x_val_lstm, y_val_lstm), 
                        batch_size = minibatch_size,
                        epochs = num_epochs,
                        verbose=1,
                        callbacks=[early_stop],
                        shuffle=False)

# saving trained model
model_path = os.path.join(cwd,'saved_model')
make_dir(model_path)
lstm_model.save(os.path.join(model_path,'DAP_Model_TLF.h5'))


/Users/natowei/ConcordNewEnergy/myenv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 24, 64)         │        17,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 64)             │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 24)             │         1,560 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 56,152 (219.34 KB)

 Trainable params: 56,152 (219.34 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
410/410 ━━━━━━━━━━━━━━━━━━━━ 10s 22ms/step - loss: 20.2824 - val_loss: 0.8730
Epoch 2/50
410/410 ━━━━━━━━━━━━━━━━━━━━ 7s 18ms/step - loss: 0.5471 - val_loss: 0.7797
Epoch 3/50
410/410 ━━━━━━━━━━━━━━━━━━━━ 8s 19ms/step - loss: 0.4758 - val_loss: 4.6985
Epoch 4/50
410/410 ━━━━━━━━━━━━━━━━━━━━ 8s 19ms/step - loss: 0.4234 - val_loss: 0.7619
Epoch 5/50
410/410 ━━━━━━━━━━━━━━━━━━━━ 8s 19ms/step - loss: 0.4650 - val_loss: 0.6820
Epoch 6/50
410/410 ━━━━━━━━━━━━━━━━━━━━ 8s 18ms/step - loss: 0.3572 - val_loss: 0.6267
Epoch 7/50
410/410 ━━━━━━━━━━━━━━━━━━━━ 8s 18ms/step - loss: 0.3439 - val_loss: 0.4687
Epoch 8/50
410/410 ━━━━━━━━━━━━━━━━━━━━ 8s 19ms/step - loss: 0.9102 - val_loss: 0.7082
Epoch 9/50
410/410 ━━━━━━━━━━━━━━━━━━━━ 8s 18ms/step - loss: 0.4677 - val_loss: 0.5783
Epoch 10/50
410/410 ━━━━━━━━━━━━━━━━━━━━ 8s 19ms/step - loss: 0.4196 - val_loss: 0.4667
Epoch 11/50
410/410 ━━━━━━━━━━━━━━━━━━━━ 9s 22ms/step - loss: 0.3374 - val_loss: 0.4519
Epoch 12/50
410/410 ━━━━━━━━━━━━━━━━━━━

In [9]:
# reload the saved model
model_path = os.path.join(cwd,'saved_model')
lstm_model = load_model(
    os.path.join(model_path, 'DAP_Model_TLF.h5'),
    custom_objects={'mse': MeanSquaredError()}
)

# Make prediciton
y_test_pred = lstm_model.predict(x_test_lstm)

# Evaluate Prediction
evaluate_prediction(y_test_pred , y_test_lstm[:,:,0], 'lstm')

# Rescale to get values before normalization
y_test_pred_rescale = y_test_pred*y_std_reg.values + y_mean_reg.values
y_test_lstm_rescale = y_test_lstm*y_std_reg.values + y_mean_reg.values

# inverse log to get prices in the actual scale
y_test_pred_invlog = np.exp(y_test_pred_rescale) -1 + min(df.loc[:,"SCED"])
y_test_lstm_invlog = np.exp(y_test_lstm_rescale) -1 + min(df.loc[:,"SCED"])

# revaluation after the rescaling
evaluate_prediction(y_test_pred_invlog , y_test_lstm_invlog[:,:,0], 'lstm')

273/273 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
MAE: 0.34
RMSE: 0.75


MAE: 34.33
RMSE: 198.11




(np.float64(34.32578416777529), np.float64(198.11097556247591))

In [11]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.losses import MeanSquaredError

# Load the model (ensure cwd and x_train_lstm are defined)
model_path = os.path.join(cwd, 'saved_model', 'DAP_Model_TLF.h5')
lstm_model = load_model(model_path, custom_objects={'mse': MeanSquaredError()})

print("LSTM model input shape:", lstm_model.input_shape)
print("LSTM model output shape:", lstm_model.output_shape)
# Expected: input shape -> (None, 24, num_features)
#           output shape -> (None, 24)

# Build the model if necessary
lstm_model.build(input_shape=(None, x_train_lstm.shape[1], x_train_lstm.shape[2]))
print("Built output shape:", lstm_model.output_shape)

# Create a sub-model that extracts the first forecast horizon (index 0)
class SliceFirst(tf.keras.layers.Layer):
    def call(self, inputs):
        return inputs[:, 0]

sub_output = tf.keras.layers.Lambda(lambda x: x[:, 0])(lstm_model.outputs[0])
sub_model = tf.keras.Model(inputs=lstm_model.inputs, outputs=sub_output)

sub_model.summary()



LSTM model input shape: (None, 24, 3)
LSTM model output shape: (None, 24)
Built output shape: (None, 24)


Model: "functional_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 24, 3)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 24, 64)         │        17,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 64)             │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 24)             │         1,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lambda (Lambda)                 │ (None)                 │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 56,152 (219.34 KB)

 Trainable params: 56,152 (219.34 KB)

 Non-trainable params: 0 (0.00 B)

In [20]:
# ------------------------
# Permutation-Based Feature Importance
# ------------------------
# Squeeze y_val_lstm if necessary (from (samples, 24, 1) to (samples, 24))
y_val_lstm_squeezed = np.squeeze(y_val_lstm, axis=-1)

# Compute baseline predictions and MSE on the validation set
baseline_pred = lstm_model.predict(x_val_lstm)
baseline_mse = np.mean((baseline_pred - y_val_lstm_squeezed)**2)
print("Baseline MSE:", baseline_mse)

# Initialize a dictionary to store importance values for each feature
feature_importance = {}
num_features = x_val_lstm.shape[2]

# Loop over each feature and measure the impact of shuffling its values
for feature in range(num_features):
    # Copy the validation data
    x_val_permuted = x_val_lstm.copy()
    # Shuffle the current feature's values for each time step across samples
    for t in range(x_val_lstm.shape[1]):
        x_val_permuted[:, t, feature] = np.random.permutation(x_val_permuted[:, t, feature])
    # Predict with the permuted data and calculate MSE
    permuted_pred = lstm_model.predict(x_val_permuted)
    permuted_mse = np.mean((permuted_pred - y_val_lstm_squeezed)**2)
    # Importance is the increase in MSE due to permutation
    importance = permuted_mse - baseline_mse
    feature_importance[feature_names[feature]] = importance

print("Feature Importance (increase in MSE):")
for feature, importance in feature_importance.items():
    print(f"{feature}: {importance:.4f}")

273/273 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Baseline MSE: 0.38616124993221385
273/273 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
273/273 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
273/273 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Feature Importance (increase in MSE):
DAP: 0.3115
SCED: -0.0053
F_TLF: 0.0415
